<a href="https://colab.research.google.com/github/ayushharshit999/Presence-Absence-of-Crackles-and-Wheezles-audio-classification-Under-Mosaic-event-Udyam-23-/blob/main/presence_of_wheezles_crackles_audio_classfication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Firstly we have to mount with the google drive to get access to train and test data.(since i have saved my text files and audio files in my google drive)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Now we have entered the data from thousands of text files into a single dictionary through file handling

In [2]:
import os
mydict = {
    'audiofile' : [],
    'patientnumber' : [],
    'recording_index' : [],
    'chest_location' : [],
    'Acquisition_mode': [],
    'recording_equipment' : [],
    'starting_time' : [],
    'ending_time' : [],
    'wheezles' : [],
    'crackles' : [],
    'class' : []
}

directory = '/content/drive/MyDrive/ICBHI_final_database/text'
for filename in os.listdir(directory):
    f = open(directory+'/'+filename,mode='r')
    bigname = f.name.split("/")
    name = bigname[6][:-4] + ".wav"
    data = f.readlines()
    for line in data:
      word = line.split()
      names = name[:-4].split('_')
      mydict['audiofile'].append(name)
      mydict['patientnumber'].append(names[0])
      mydict['recording_index'].append(names[1])
      mydict['chest_location'].append(names[2])
      mydict['Acquisition_mode'].append(names[3])
      mydict['recording_equipment'].append(names[4])
      mydict['starting_time'].append(word[0])
      mydict['ending_time'].append(word[1])
      mydict['wheezles'].append(word[2])
      mydict['crackles'].append(word[3])
      if word[2]=='0' and word[3]=='0':
        mydict['class'].append(0)
      elif word[2]=='1' and word[3]=='0':
        mydict['class'].append(1)
      elif word[2]=='0' and word[3]=='1':
        mydict['class'].append(2)
      elif word[2]=='1' and word[3]=='1':
        mydict['class'].append(3)
    f.close()

In [3]:
print(len(mydict['class']))
print(len(mydict['wheezles']))

6110
6110


Now we have converted the dictionary into dataframe through pandas library

In [4]:
import pandas as pd
df = pd.DataFrame(mydict)
df.head()

,audiofile,patientnumber,recording_index,chest_location,Acquisition_mode,recording_equipment,starting_time,ending_time,wheezles,crackles,class
0,107_2b3_Pr_mc_AKGC417L.wav,107,2b3,Pr,mc,AKGC417L,0.887,2.601,0,0,0
1,107_2b3_Pr_mc_AKGC417L.wav,107,2b3,Pr,mc,AKGC417L,2.601,5.327,1,0,1
2,107_2b3_Pr_mc_AKGC417L.wav,107,2b3,Pr,mc,AKGC417L,5.327,7.923,1,1,3
3,107_2b3_Pr_mc_AKGC417L.wav,107,2b3,Pr,mc,AKGC417L,7.923,10.53,0,0,0
4,107_2b3_Pr_mc_AKGC417L.wav,107,2b3,Pr,mc,AKGC417L,10.53,13.196,1,0,1


In [5]:
df.dtypes

audiofile              object
patientnumber          object
recording_index        object
chest_location         object
Acquisition_mode       object
recording_equipment    object
starting_time          object
ending_time            object
wheezles               object
crackles               object
class                   int64
dtype: object

Since we have to use these columns as float values for audio slicing later.So we have converted them to float values

In [6]:
df['starting_time'] = df['starting_time'].astype(float)
df['ending_time'] = df['ending_time'].astype(float)
df['class'] = df['class'].astype(float)

In [7]:
df.dtypes

audiofile               object
patientnumber           object
recording_index         object
chest_location          object
Acquisition_mode        object
recording_equipment     object
starting_time          float64
ending_time            float64
wheezles                object
crackles                object
class                  float64
dtype: object

 **Audio Slicing and preprocessing via librosa and pydub**

In [8]:
!pip install pydub

In [9]:
from pydub import AudioSegment
audio_file_path='/content/drive/MyDrive/ICBHI_final_database/audio'

Created a function via librosa library to convert audio file into an array of 40 integers.So that we can later train our model on it

In [10]:
import librosa
import numpy as np
def features_extractor(file_name):
    audio, sample_rate = librosa.load(file_name)
    mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

    return mfccs_scaled_features

In [11]:
!pip install librosa

Code to slice audio according to starting and ending time

In [12]:
extracted_features=[]
from tqdm import tqdm
extracted_features=[]
for index_num,row in tqdm(df.iterrows()):
    file_name = audio_file_path+'/'+str(row["audiofile"])
    playaudio = AudioSegment.from_file(file_name, format="wav")
    sliceaudio = playaudio[row["starting_time"]*1000:row["ending_time"]*1000]
    sliceaudio.export(out_f="outNow.wav",format="wav")

    final_class_labels=row["class"]
    # print(audio_file_path)
    # print(row["audiofile"])
    data=features_extractor("outNow.wav")
    extracted_features.append([data,final_class_labels])


6110it [43:08,  2.36it/s]


Created a new dataframe

In [13]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['feature','class'])
extracted_features_df.head()

,feature,class
0,"[-229.83647, 139.52625, 58.618626, 41.506653, ...",0.0
1,"[-244.32118, 128.84822, 62.34122, 49.60628, 27...",1.0
2,"[-229.44312, 146.91884, 68.53155, 46.53205, 20...",3.0
3,"[-234.47693, 141.27098, 66.164696, 47.675743, ...",0.0
4,"[-230.46887, 145.73125, 65.75772, 45.434685, 1...",1.0


Since we are applying neural network model so we converted it into numpy array

In [14]:
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [15]:
X.shape

(6110, 40)

In [16]:
y.shape

(6110,)

Normalized the dataset to make neural network work more efficiently

In [82]:
normed_x = X/ X.sum(axis=1)[:,np.newaxis]

Splitting the data into train and test

In [83]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(normed_x,y,test_size=0.2,random_state=0)

In [18]:
import tensorflow as tf
print(tf.__version__)

2.12.0


Creating the Model

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense,Dropout,Activation
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [84]:
model = Sequential()

# Input layer
model.add(Dense(40, kernel_initializer='normal',input_dim = X.shape[1], activation='relu'))

# Hidden Layers
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))
model.add(Dense(256, kernel_initializer='normal',activation='relu'))

# Output Layers
model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compilation and Summary
model.compile(optimizer = 'adam',loss='mean_absolute_error',metrics =['accuracy'])
model.summary()



Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_117 (Dense)           (None, 40)                1640      
                                                                 
 dense_118 (Dense)           (None, 256)               10496     
                                                                 
 dense_119 (Dense)           (None, 256)               65792     
                                                                 
 dense_120 (Dense)           (None, 256)               65792     
                                                                 
 dense_121 (Dense)           (None, 1)                 257       
                                                                 
Total params: 143,977
Trainable params: 143,977
Non-trainable params: 0
_________________________________________________________________


In [85]:
num_epochs = 1000
num_batch_size = 32
model.fit(X_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(X_test, y_test), verbose=1)

Epoch 1/1000
153/153 [==============================] - 3s 6ms/step - loss: 0.7884 - accuracy: 0.5178 - val_loss: 0.7680 - val_accuracy: 0.5172
Epoch 2/1000
153/153 [==============================] - 1s 5ms/step - loss: 0.8178 - accuracy: 0.5149 - val_loss: 0.7187 - val_accuracy: 0.4910
Epoch 3/1000
153/153 [==============================] - 1s 5ms/step - loss: 0.7246 - accuracy: 0.4906 - val_loss: 0.7003 - val_accuracy: 0.4885
Epoch 4/1000
153/153 [==============================] - 1s 4ms/step - loss: 0.7044 - accuracy: 0.5176 - val_loss: 0.7412 - val_accuracy: 0.5229
Epoch 5/1000
153/153 [==============================] - 1s 4ms/step - loss: 0.6955 - accuracy: 0.5266 - val_loss: 0.6868 - val_accuracy: 0.5516
Epoch 6/1000
153/153 [==============================] - 1s 6ms/step - loss: 0.6930 - accuracy: 0.5297 - val_loss: 0.6812 - val_accuracy: 0.5262
Epoch 7/1000
153/153 [==============================] - 1s 7ms/step - loss: 0.6828 - accuracy: 0.5299 - val_loss: 0.7116 - val_accuracy:

In [91]:
test_accuracy=model.evaluate(X_test,y_test,verbose=0)
print(test_accuracy[1])

0.6153846383094788


In [92]:
model.save_weights("model.h5")